In [49]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [72]:
dtypes = {'ISBN': 'str', 'Book-Title': 'str', 'Book-Author': 'str', 'Year-Of-Publication': 'str', 'Publisher': 'str', 'Image-URL-S': 'str', 'Image-URL-M': 'str', 'Image-URL-L': 'str'}

# books_df = pd.read_csv('dataset/Books.csv', dtype=dtypes)
# users_df = pd.read_csv('dataset/Users.csv')
ratings_df = pd.read_csv('dataset/Ratings.csv')
# movie_ratings_df = pd.read_csv('dataset/MovieRatings.csv')
ratings_df['Book-Rating'] = ratings_df['Book-Rating'] / 2


cpu


In [51]:
# df = ratings_df.merge(books_df, how="left", on="ISBN")
# df.head().to_csv('dataset/test.csv')

In [52]:
ratings_df.info()
ratings_df.head



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   User-ID      1149780 non-null  int64  
 1   ISBN         1149780 non-null  object 
 2   Book-Rating  1149780 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 26.3+ MB


<bound method NDFrame.head of          User-ID         ISBN  Book-Rating
0         276725   034545104X          0.0
1         276726   0155061224          2.5
2         276727   0446520802          0.0
3         276729   052165615X          1.5
4         276729   0521795028          3.0
...          ...          ...          ...
1149775   276704   1563526298          4.5
1149776   276706   0679447156          0.0
1149777   276709   0515107662          5.0
1149778   276721   0590442449          5.0
1149779   276723  05162443314          4.0

[1149780 rows x 3 columns]>

In [53]:
# Create a boolean mask that is True for rows that don't have a Book-Rating of 0
mask = ratings_df['Book-Rating'] != 0

# Use boolean indexing to select only the rows that don't have a Book-Rating of 0
ratings_df = ratings_df[mask]
ratings_df.head


<bound method NDFrame.head of          User-ID         ISBN  Book-Rating
1         276726   0155061224          2.5
3         276729   052165615X          1.5
4         276729   0521795028          3.0
6         276736   3257224281          4.0
7         276737   0600570967          3.0
...          ...          ...          ...
1149773   276704   0806917695          2.5
1149775   276704   1563526298          4.5
1149777   276709   0515107662          5.0
1149778   276721   0590442449          5.0
1149779   276723  05162443314          4.0

[433671 rows x 3 columns]>

In [54]:
from collections import Counter

lbl_user = preprocessing.LabelEncoder()
lbl_book = preprocessing.LabelEncoder()
ratings_df['User-ID'] = lbl_user.fit_transform(ratings_df['User-ID'].values)
ratings_df['ISBN'] = lbl_book.fit_transform(ratings_df['ISBN'].values)

user_ratings_count = Counter(ratings_df['User-ID'])

# Find users with less than 4 ratings
users_to_remove = [user_id for user_id, count in user_ratings_count.items() if count < 10]

# Remove users with less than 4 ratings from the dataset
ratings_df = ratings_df[~ratings_df['User-ID'].isin(users_to_remove)]
ratings_df.head



C:\Users\youssef albali\AppData\Local\Temp\ipykernel_8436\1473222708.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_df['User-ID'] = lbl_user.fit_transform(ratings_df['User-ID'].values)
C:\Users\youssef albali\AppData\Local\Temp\ipykernel_8436\1473222708.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_df['ISBN'] = lbl_book.fit_transform(ratings_df['ISBN'].values)


<bound method NDFrame.head of          User-ID    ISBN  Book-Rating
133        77209    2884          5.0
134        77209   15804          4.5
135        77209   16137          5.0
136        77209   17027          4.5
137        77209   39925          4.5
...          ...     ...          ...
1149743    77175  117346          5.0
1149744    77175  117457          5.0
1149745    77175  125788          5.0
1149746    77175  134545          3.0
1149747    77175  141488          3.5

[295561 rows x 3 columns]>

In [55]:
train_df, valid_df = train_test_split(
    ratings_df, test_size=0.1, stratify=ratings_df['Book-Rating'].values
)

# train_df.to_csv('dataset/test.csv')


In [56]:
valid_df.shape

(29557, 3)

In [57]:
from bookDataset import BookDataset

# Create train and validation datasets
train_dataset = BookDataset(train_df['User-ID'].values, train_df['ISBN'].values, train_df['Book-Rating'].values)
valid_dataset = BookDataset(valid_df['User-ID'].values, valid_df['ISBN'].values, valid_df['Book-Rating'].values)

# Create train and validation data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, num_workers=4, drop_last=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=64, shuffle=True, num_workers=4, drop_last=True)

266004
266004
266004
266004


In [58]:
class BookRecommender(torch.nn.Module):
    def __init__(self, num_users, num_isbns, embedding_dim):
        super(BookRecommender, self).__init__()
        self.user_embedding = torch.nn.Embedding(num_embeddings=num_users, embedding_dim=embedding_dim)
        self.isbn_embedding = torch.nn.Embedding(num_embeddings=num_isbns, embedding_dim=embedding_dim)
        self.fc1 = torch.nn.Linear(embedding_dim * 2, 64)
        self.fc2 = torch.nn.Linear(64, 32)
        self.fc3 = torch.nn.Linear(32, 1)

    def forward(self, users, isbns):
        user_embeds = self.user_embedding(users.long())
        isbn_embeds = self.isbn_embedding(isbns.long())
        embeds = torch.cat([user_embeds, isbn_embeds], dim=1)
        x = torch.relu(self.fc1(embeds.view(embeds.size(0), -1)))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [59]:
model = BookRecommender(num_users=len(lbl_user.classes_),
                        num_isbns=len(lbl_book.classes_),
                        embedding_dim=64)

print(model)

BookRecommender(
  (user_embedding): Embedding(77805, 64)
  (isbn_embedding): Embedding(185973, 64)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
)


In [60]:
import torch.optim as optim

learning_rate = 0.1
num_epochs = 2
batch_size = 64

# Define loss function and optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.7)
criterion = torch.nn.MSELoss()

In [61]:
from bookDataset import BookDataset
num_batches = len(train_loader)

print("Number of batches in the DataLoader:", num_batches)

for epoch in range(num_epochs):
    running_loss = 0.0
    for batch in train_loader:
        # zero the parameter gradients
        optimizer.zero_grad()
        outputs = model(batch["user_id"], batch["isbn"])
        loss = criterion(outputs, batch["rating"].unsqueeze(1))
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item() * batch_size
    epoch_loss = running_loss / len(train_dataset)
    print("Learning Rate:", scheduler.get_last_lr())
    
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, epoch_loss))
    scheduler.step()
print('Finished Training')

Number of batches in the DataLoader: 4156
Learning Rate: [0.1]
Epoch [1/2], Loss: 0.7994
Learning Rate: [0.1]
Epoch [2/2], Loss: 0.7299
Finished Training


In [79]:
from sklearn.metrics import mean_squared_error

total_loss = 0.0
total_correct = 0
total_samples = 0
print('hello', batch_size)
predictions = []
targets = []
model.eval()

with torch.no_grad():
    for batch in valid_loader:
        user_ids, isbns, ratings = batch['user_id'], batch['isbn'], batch['rating']
        outputs = model(user_ids, isbns)
        ratings = ratings.view(-1, 1)  # Reshape the target tensor
        predicted_ratings = torch.round(outputs)  # Round the predicted ratings
#         print(ratings.shape)
        predictions.extend(predicted_ratings.tolist())
        targets.extend(ratings.tolist())
    
        correct = ((predicted_ratings == ratings) | (predicted_ratings == ratings - 0.5) | (predicted_ratings == ratings + 0.5)).sum().item()
        total_correct += correct
        total_samples += ratings.size(0)

# Calculate mean squared error
rmse = mean_squared_error(targets, predictions)
print('Validation RMSE: {:.2f}'.format(rmse))

# Calculate accuracy
accuracy = total_correct / total_samples
print('Validation Accuracy: {:.2f}%'.format(accuracy * 100))


hello 64
Validation RMSE: 0.81
Validation Accuracy: 60.31%
